In [1]:
import pandas as pd
import folium
from folium.plugins import HeatMap
import os
import ipywidgets as widgets
from IPython.display import display, clear_output

In [2]:

df = pd.read_csv('../data/homicide_cleaned.csv', parse_dates=['Incident_Date'])
df = df.dropna(subset=['Latitude', 'Longitude'])

def plot_heatmap(df_subset):
    if df_subset.empty:
        return folium.Map(location=[35.2271, -80.8431], zoom_start=12)
    heat_data = df_subset[['Latitude', 'Longitude']]. dropna().values.tolist()
    m = folium.Map(location=[df_subset['Latitude'].mean(), df_subset['Longitude'].mean()],
    zoom_start=11, tiles='OpenStreetMap')
    HeatMap(heat_data, radius=20, blur=15, min_opacity=0.45, max_zoom=16).add_to(m)
    return m

weapon_options = sorted(df['Weapon'].dropna().unique())
weapon_dropdown = widgets.Dropdown(
    options=['All'] + sorted(df['Weapon'].dropna().unique().tolist()),
    value='All',
    description='Weapon:',
    style={'description_width': 'initial'}
)

year_dropdown = widgets.Dropdown(
    options=['All'] + sorted(df['Year'].dropna().unique().tolist()),
    value='All',
    description='Year:'
)
month_labels = {
    1: 'January', 2: 'February', 3: 'March', 4: 'April', 5: 'May', 6: 'June', 7: 'July',
    8: 'August', 9: 'September', 10: 'October', 11: 'November', 12: 'December'
}

month_dropdown = widgets.Dropdown(
    options=[('All', 'All')] + [(label, num) for num, label in month_labels.items()],
    value='All',
    description='Month:'
)

weekday_labels = {
    0: 'Monday', 1:'Tuesday', 2: 'Wednesday', 3: 'Thursday',
    4: 'Friday', 5: 'Saturday', 6: 'Sunday'
}
weekday_dropdown = widgets.Dropdown(
    options=[('All', 'All')] + [(label, num) for num, label in weekday_labels.items()],
    value='All',
    description='Weekday:'
)

output = widgets.Output()

def update_map(change=None):
    year = year_dropdown.value
    month = month_dropdown.value
    weekday = weekday_dropdown.value
    weapon = weapon_dropdown.value
    filtered_df = df.copy()
    if year != 'All':
        filtered_df = filtered_df[filtered_df['Year'] == year]
    if month != 'All':
        filtered_df = filtered_df[filtered_df['Month'] == month]
    if weekday != 'All':
        filtered_df = filtered_df[filtered_df['Weekday'] == weekday]
    if weapon != 'All':
        filtered_df = filtered_df[filtered_df['Weapon'] == weapon]
    m = plot_heatmap(filtered_df)
    with output:
        clear_output(wait=True)
        if filtered_df.empty:
            print("No incidents found for this combination.")
        display(m)

year_dropdown.observe(update_map, names='value')
month_dropdown.observe(update_map, names='value')
weekday_dropdown.observe(update_map, names='value')
weapon_dropdown.observe(update_map, names='value')

display(widgets.VBox([
    year_dropdown,
    month_dropdown,
    weekday_dropdown,
    weapon_dropdown
]), output)

update_map()

Output()